# Problem 1: Generation of random graphs of a given degree distribution

Here we define the Graph class and auxiliary functions to be used in the rest of the problems.

The best Python data structure for this task seems to be the dictionary, because it can hold multiple values corresponding to a single key, just like a dictionary can have multiple explanations for a word.

Here the "key" is the vertex number, and its "values" are the vertices it is connected to.

In [ ]:
import numpy as np
from scipy.optimize import fsolve
import sympy
import matplotlib.pyplot as plt
import copy

class Graph(object):

    def __init__(self, graph_dict=None): 
        if graph_dict == None:
            graph_dict = {}
        self.graph_dict = graph_dict
        self.visited = np.zeros(len(graph_dict))
        self.components = []
        
    def vertices(self):
        return list(self.graph_dict.keys())

    def edges(self):
        return self.generate_edges()

    def add_vertex(self, vertex):
        if vertex not in self.graph_dict:
            self.graph_dict[vertex] = []

    def add_edge(self, edge):
        edge = set(edge)
        (vertex1, vertex2) = tuple(edge)
        if vertex1 in self.graph_dict:
            self.graph_dict[vertex1].append(vertex2)
        else:
            self.graph_dict[vertex1] = [vertex2]
        if vertex2 in self.graph_dict:
            self.graph_dict[vertex2].append(vertex1)
        else:
            self.graph_dict[vertex2] = [vertex1]

    def generate_edges(self):
        edges = []
        for vertex in self.graph_dict:
            for neighbour in self.graph_dict[vertex]:
                if {neighbour, vertex} not in edges:
                    edges.append({vertex, neighbour})
        return edges


    def __str__(self):
        res = "vertices: "
        for k in self.graph_dict:
            res += str(k) + " "
        res += "\nedges: "
        for edge in self.generate_edges():
            res += str(edge) + " "
        return res           
        
    
    def find_components(self):
        def dfs(node, component, self):
            for neigh in self.graph_dict[node]:
                if self.visited[neigh] == 0:
                    self.visited[neigh] = 1
                    component.append(neigh)
                    dfs(neigh, component, self)
                
        for node in self.graph_dict:
            if self.visited[node] == 0:
                component = []
                self.visited[node] = 1
                component.append(node)
                dfs(node, component, self)
                self.components.append(component)
        self.largest_component = len(max(self.components, key=len))
        
        
    def start_spins(self):
        self.spins = []
        for i in range(len(self.graph_dict)):
            self.spins.append(np.random.choice([-1, 1]))


    def start_fields(self):
        self.field_dict = {}
        for node in self.graph_dict:
            self.field_dict[node] = [np.random.uniform(0, 0.01) for neigh in self.graph_dict[node]]
            
            
    def propagate(self, T):
        idx = np.random.choice(list(self.graph_dict.keys()))
        neighs = []
        for n in self.graph_dict[idx]:
            neighs.append(n)
        for j in range(len(neighs)):
            n_j = neighs[j] # neighbour j
            new_h_ji = 0
            for h_kj in self.field_dict[n_j]:
                new_h_ji += T/2 * np.log(np.cosh((1 + h_kj)/T) / np.cosh((-1 + h_kj)/T))
            self.field_dict[idx][j] = new_h_ji
        
        
def connect(halfedges, graph):
    while len(halfedges) > 1:
        edge1 = np.random.choice(halfedges)
        temp = halfedges[:]
        temp.remove(edge1)
        edge2 = np.random.choice(temp)
        if edge1 != edge2:
            if {edge1, edge2} not in graph.edges(): 
                graph.add_edge([edge1, edge2])
        else:
            if len(halfedges) < 3:
                break    
            else:
                if halfedges[1:] == halfedges[:-1]:
                    break
            continue
        halfedges.remove(edge1)
        halfedges.remove(edge2)
        

def generate(N, pi):
        init_halfedges = []
        graph = {}
        for i in range(N):
            graph[i] = []
            if np.random.uniform() < pi:
                init_halfedges.append(4*[i])
            else:
                init_halfedges.append([i])
        halfedges = [item for sublist in init_halfedges for item in sublist]
        graph = Graph(graph)
        connect(halfedges, graph)
        return graph


def q_core(graph, q, deg_less_q = []):
    temp_graph = copy.deepcopy(graph)
    if len(deg_less_q) == 0:
        for node in graph.graph_dict:
            if 0 < len(graph.graph_dict[node]) < q:
                deg_less_q.append(node)
    
    if len(deg_less_q) == 0:
        return graph
    node_to_remove = np.random.choice(deg_less_q)
    neighbours = [neigh for neigh in temp_graph.graph_dict[node_to_remove]]
    temp_graph.graph_dict[node_to_remove] = []
    
    for neigh in neighbours:
        temp_graph.graph_dict[neigh].remove(node_to_remove)
        if len(temp_graph.graph_dict[neigh]) < q:
            deg_less_q.append(neigh)
    deg_less_q.remove(node_to_remove)
    
    return q_core(temp_graph, q, deg_less_q)



def flip(graph, T):
    to_flip = np.random.choice(list(graph.graph_dict.keys()))
    neighs = []
    for n in graph.graph_dict[to_flip]:
        neighs.append(n)
    old_E = 0
    new_E = 0
    for n in neighs:
        old_E += - graph.spins[to_flip] * graph.spins[n]
        new_E +=  graph.spins[to_flip] * graph.spins[n]
    proba = min(1, np.exp(-(new_E - old_E) / T))
    
    if proba >= np.random.uniform():
        graph.spins[to_flip] = -graph.spins[to_flip]



def magnetization(graph):
    return np.sum(graph.spins) / len(graph.graph_dict)








